<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [46]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("POL_1.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,indexed_at,provider_code,series_code,series_name,series_num,ISO3
0,1989,1989-01-01,NA,annual,APRI,PRICE INDICES,A,2022-04-04T17:35:14.723Z,STATPOL,APRI-10,Price indices of consumer goods and services -...,Master,POL
1,1990,1990-01-01,NA,annual,APRI,PRICE INDICES,A,2022-04-04T17:35:14.723Z,STATPOL,APRI-10,Price indices of consumer goods and services -...,Master,POL
2,1991,1991-01-01,NA,annual,APRI,PRICE INDICES,A,2022-04-04T17:35:14.723Z,STATPOL,APRI-10,Price indices of consumer goods and services -...,Master,POL
3,1992,1992-01-01,NA,annual,APRI,PRICE INDICES,A,2022-04-04T17:35:14.723Z,STATPOL,APRI-10,Price indices of consumer goods and services -...,Master,POL
4,1993,1993-01-01,NA,annual,APRI,PRICE INDICES,A,2022-04-04T17:35:14.723Z,STATPOL,APRI-10,Price indices of consumer goods and services -...,Master,POL


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [47]:
python_df = pd.read_pickle("POL_1.pickle")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQ,Frequency
0,annual,STATPOL,ABOP,BALANCE OF PAYMENTS,ABOP-10,Current account of the balance of payments - b...,2000,2000-01-01,NA,NaN,A,Annually
1,annual,STATPOL,ABOP,BALANCE OF PAYMENTS,ABOP-10,Current account of the balance of payments - b...,2001,2001-01-01,NA,NaN,A,Annually
2,annual,STATPOL,ABOP,BALANCE OF PAYMENTS,ABOP-10,Current account of the balance of payments - b...,2002,2002-01-01,NA,NaN,A,Annually
3,annual,STATPOL,ABOP,BALANCE OF PAYMENTS,ABOP-10,Current account of the balance of payments - b...,2003,2003-01-01,NA,NaN,A,Annually
4,annual,STATPOL,ABOP,BALANCE OF PAYMENTS,ABOP-10,Current account of the balance of payments - b...,2004,2004-01-01,-6483.0,-6483.0,A,Annually


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [48]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [49]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [50]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [51]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

False

<font size=5, font color='blue'> Stata_df fails to download ['value'] for AFT and AGG series while python_df succeeds

In [42]:
diffs = python_sub.compare(stata_sub)
diff_index_labels = diffs.index.unique().tolist()
print(diff_index_labels)


[1, 6, 7, 9, 10, 12, 13, 15, 18, 171, 176, 177, 179, 180, 182, 183, 185, 188, 341, 346, 347, 349, 350, 352, 353, 355, 358, 511, 516, 517, 519, 520, 522, 523, 525, 528, 681, 686, 687, 689, 690, 692, 693, 695, 697, 698, 851, 856, 857, 859, 860, 862, 863, 865, 867, 868, 1021, 1026, 1027, 1029, 1030, 1032, 1033, 1035, 1036, 1037, 1038, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1432, 1437, 1438, 1440, 1441, 1443, 1444, 1446, 1447, 1448, 1449, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1843, 1848, 1849, 1851, 1852, 1854, 1855, 1857, 1858, 1859, 1860, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 2254, 2259, 2260, 2262, 2263, 2265, 2266, 2268, 2269, 2270, 2271, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2665, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 2678, 2679, 2680, 2681, 2682, 2686, 2687, 2688, 2689, 2690, 2691, 2692, 2693, 2694, 2695, 3094, 3095, 3096, 3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 

In [43]:
python_sub.iloc[diff_index_labels]

,period_start_day,series_code,value
1,1989-01-01,AFT-11,101.5
6,1989-01-01,AFT-16,19476.2
7,1989-01-01,AFT-17,13466.1
9,1989-01-01,AFT-19,15416.1
10,1989-01-01,AFT-20,10277.3
...,...,...,...
12840,2020-01-01,AGG-15,57.4
12841,2020-01-01,AGG-6,-166087.0
12842,2020-01-01,AGG-7,-7.1
12843,2020-01-01,AGG-8,-187979.0


In [44]:
stata_sub.iloc[diff_index_labels]

,period_start_day,series_code,value
1,1989-01-01,AFT-11,NaN
6,1989-01-01,AFT-16,NaN
7,1989-01-01,AFT-17,NaN
9,1989-01-01,AFT-19,NaN
10,1989-01-01,AFT-20,NaN
...,...,...,...
12840,2020-01-01,AGG-15,NaN
12841,2020-01-01,AGG-6,NaN
12842,2020-01-01,AGG-7,NaN
12843,2020-01-01,AGG-8,NaN


<font size=5, font color='blue'> Compare the rows in stata_df without missing values to the corresponding rows in Python

In [58]:
stata_not_na = stata_sub[stata_sub.notna().all(axis=1)].index

In [59]:
stata_sub.loc[stata_not_na].equals(python_sub.loc[stata_not_na])

True